# Part 2: Streaming application using Spark Structured Streaming  
In this task, you will implement Spark Structured Streaming to consume the data from task 1 and perform a prediction.    
Important:   
-	This task uses PySpark Structured Streaming with PySpark Dataframe APIs and PySpark ML.  
-	You also need your pipeline model from A2A to make predictions and persist the results.  

1.	Write code to create a SparkSession, which 1) uses four cores with a proper application name; 2) use the Melbourne timezone; 3) ensure a checkpoint location has been set.


In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'

# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[4]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Assignment2B"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name) \
                        .set("spark.sql.streaming.checkpointLocation", "checkpoints")

# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).config("spark.sql.session.timeZone", "GMT+10").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

from pyspark.sql import functions as F

2.	Write code to define the data schema for the data files, following the data types suggested in the metadata file. Load the static datasets (e.g. building information) into data frames. (You can reuse your code from 2A.)


In [ ]:
# Adapted from GPT
from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, StringType, DecimalType, TimestampType
)

# 1. Meters Table
meters_schema = StructType([
    StructField("building_id", IntegerType(), False),
    StructField("meter_type", StringType(), False),   # Char(1) -> StringType
    StructField("ts", TimestampType(), False),
    StructField("value", DecimalType(15, 4), False),
    StructField("row_id", IntegerType(), False)
])

# 2. Buildings Table
buildings_schema = StructType([
    StructField("site_id", IntegerType(), False),
    StructField("building_id", IntegerType(), False),
    StructField("primary_use", StringType(), True),
    StructField("square_feet", IntegerType(), True),
    StructField("floor_count", IntegerType(), True),
    StructField("row_id", IntegerType(), False),
    StructField("year_built", IntegerType(), True),
    StructField("latent_y", DecimalType(6, 4), True),
    StructField("latent_s", DecimalType(6, 4), True),
    StructField("latent_r", DecimalType(6, 4), True)
])

# 3. Weather Table
# weather_schema = StructType([
#     StructField("site_id", StringType(), False),
#     StructField("timestamp", StringType(), False),
#     StructField("air_temperature", StringType(), True),
#     StructField("cloud_coverage", StringType(), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
#     StructField("dew_temperature", StringType(), True),
#     StructField("sea_level_pressure", StringType(), True),
#     StructField("wind_direction", StringType(), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
#     StructField("wind_speed", StringType(), True),
#     StructField("weather_ts", StringType(), False) # new field
# ])

weather_schema = StructType([
    StructField("site_id", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("air_temperature", DecimalType(5, 3), True),
    StructField("cloud_coverage", DecimalType(5, 3), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
    StructField("dew_temperature", DecimalType(5, 3), True),
    StructField("sea_level_pressure", DecimalType(8, 3), True),
    StructField("wind_direction", DecimalType(5, 3), True), # Is an Integer, but ends with a ".0", so read as a DecimalType
    StructField("wind_speed", DecimalType(5, 3), True),
    StructField("weather_ts", TimestampType(), False) # new field
])

buildings_df = spark.read.csv(
    "data/new_building_information.csv",
    header=True,
    schema=buildings_schema
)

weather_df = spark.read.csv(
    "data/weather.csv",
    header=True,
    schema=weather_schema
)


3.	Using the Kafka topic from the producer in Task 1, ingest the streaming data into Spark Streaming, assuming all data comes in the String format. Except for the 'weather_ts' column, you shall receive it as an Int type. Load the new building information CSV file into a dataframe. Then, the data frames should be transformed into the proper formats following the metadata file schema, similar to assignment 2A.


In [ ]:


#configuration
hostip = "192.168.0.6"
topic = 'weather_data'

df_raw = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", f'{hostip}:9092') \
    .option("subscribe", topic) \
    .load()

df_str = df_raw.selectExpr("CAST(value AS STRING) as json_str")

weather_stream = (
    df_str
    .withColumn("data", F.from_json(F.col("json_str"), F.ArrayType(weather_schema)))
    .select(F.explode(F.col("data")).alias("r"))
    .select("r.*")
)


4.	Use a watermark on weather_ts, if data points are received 5 seconds late, discard the data.

In [4]:
weather_stream = weather_stream.withWatermark("weather_ts", '5 seconds')

5.	Perform the necessary transformation you used in A2A. (note: every student may have used different features, feel free to reuse the code you have written in A2A. If you built an end-to-end pipeline, you can ignore this task.) 

In [ ]:

# Get global_means, site_month_means, site_means
# weather_df is history, weather_stream is current

# Split timestamp to date, month, time bucket
weather_df = weather_df.withColumn("date", F.to_date("timestamp")).withColumn(
    "time",
    F.when(F.hour("timestamp") <= 5, "0-6h")
     .when(F.hour("timestamp") <= 11, "6-12h")
     .when(F.hour("timestamp") <= 17, "12-18h")
     .when(F.hour("timestamp") <= 23, "18-24h")
).withColumn("month", F.month("timestamp"))

# Choose which columns to impute
impute_cols = [
    "air_temperature",
    "cloud_coverage",
    "dew_temperature",
    "sea_level_pressure",
    "wind_direction",
    "wind_speed"
]

# Compute global_means, site_month_means, site_means
global_means = weather_df.select(
    *[F.mean(c).alias(c) for c in impute_cols]
).first().asDict()

site_month_means = weather_df.groupBy("site_id", "month").agg(
    *[F.mean(c).alias(f"{c}_site_month_mean") for c in impute_cols]
)

site_means = weather_df.groupBy("site_id").agg(
    *[F.mean(c).alias(f"{c}_site_mean") for c in impute_cols]
)
    
# Skip Garbage collection
# del site_month_means
# del site_means
# del global_means
# spark.catalog.clearCache()

# Data imputation
# Transform weather_stream
# Split timestamp to date, month, time bucket
weather_stream = weather_stream.withColumn("date", F.to_date("timestamp")).withColumn(
    "time",
    F.when(F.hour("timestamp") <= 5, "0-6h")
     .when(F.hour("timestamp") <= 11, "6-12h")
     .when(F.hour("timestamp") <= 17, "12-18h")
     .when(F.hour("timestamp") <= 23, "18-24h")
).withColumn("month", F.month("timestamp"))


# Step 1: site_id + month
weather_stream = weather_stream.join(site_month_means, on=["site_id", "month"], how="left")
for c in impute_cols:
    weather_stream = weather_stream.withColumn(
        c, F.coalesce(c, F.col(f"{c}_site_month_mean"))
    ).drop(f"{c}_site_month_mean")
    
# Step 2: site_id
weather_stream = weather_stream.join(site_means, on="site_id", how="left")
for c in impute_cols:
    weather_stream = weather_stream.withColumn(
        c, F.coalesce(c, F.col(f"{c}_site_mean"))
    ).drop(f"{c}_site_mean")

# Step 3: global fallback
for c in impute_cols:
    weather_stream = weather_stream.withColumn(
        c, F.coalesce(c, F.lit(global_means[c]))
    )
# Aggregate by time bucket
weather_stream = (
    weather_stream
    .groupBy(
        "site_id", "date", "time", "month",
        F.window("weather_ts", "5 seconds")
    )
    .agg(
        F.mean("air_temperature").cast(DecimalType(5, 3)).alias("air_temperature"),
        F.mean("cloud_coverage").cast(DecimalType(5, 3)).alias("cloud_coverage"),
        F.mean("dew_temperature").cast(DecimalType(5, 3)).alias("dew_temperature"),
        F.mean("sea_level_pressure").cast(DecimalType(8, 3)).alias("sea_level_pressure"),
        F.mean("wind_direction").cast(DecimalType(5, 3)).alias("wind_direction"),
        F.mean("wind_speed").cast(DecimalType(5, 3)).alias("wind_speed")     
    )    
)

# Add custom columns
weather_stream = (
    weather_stream
    .withColumn("dew_depression", F.col("air_temperature") - F.col("dew_temperature"))
    .withColumn("nonideal_temp", (F.col("air_temperature") - 18)**2)
    .drop("air_temperature")
    .drop("dew_temperature")
)

# No need to add median temp and peak-offpeak as our pipeline model later does not use them
feature_df = buildings_df.join(weather_stream, ["site_id"])


6.	Load your pipeline model and perform the following aggregations:  
a)	Print the prediction from your model as a stream comes in.  
b)	Every 7 seconds, print the total energy consumption for each 6-hour interval, aggregated by building, and print 20 records. (Note: This is simulating energy data each day in a week)  
c)	Every 14 seconds, for each site, print the daily total energy consumption.  

In [7]:

# --- 1. Spark + model setup ---
# from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

model = PipelineModel.load("models/best_model_rmsle")

# --- 2. Apply model ---
predictions = model.transform(feature_df).withColumnRenamed("prediction", "log_power_usage")

checkpoint_dir = os.path.abspath("checkpoints/weather_stream")
os.makedirs(checkpoint_dir, exist_ok=True)

In [8]:
weather_stream.printSchema()

root
 |-- site_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- cloud_coverage: decimal(5,3) (nullable = true)
 |-- sea_level_pressure: decimal(8,3) (nullable = true)
 |-- wind_direction: decimal(5,3) (nullable = true)
 |-- wind_speed: decimal(5,3) (nullable = true)
 |-- dew_depression: decimal(6,3) (nullable = true)
 |-- nonideal_temp: double (nullable = true)



In [9]:
feature_df.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: integer (nullable = true)
 |-- floor_count: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- latent_y: decimal(6,4) (nullable = true)
 |-- latent_s: decimal(6,4) (nullable = true)
 |-- latent_r: decimal(6,4) (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- cloud_coverage: decimal(5,3) (nullable = true)
 |-- sea_level_pressure: decimal(8,3) (nullable = true)
 |-- wind_direction: decimal(5,3) (nullable = true)
 |-- wind_speed: decimal(5,3) (nullable = true)
 |-- dew_depression: decimal(6,3) (nullable = true)
 |-- nonideal_temp: double (nullable = true)



In [10]:
predictions.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: integer (nullable = true)
 |-- floor_count: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- latent_y: decimal(6,4) (nullable = true)
 |-- latent_s: decimal(6,4) (nullable = true)
 |-- latent_r: decimal(6,4) (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- cloud_coverage: decimal(5,3) (nullable = true)
 |-- sea_level_pressure: decimal(8,3) (nullable = true)
 |-- wind_direction: decimal(5,3) (nullable = true)
 |-- wind_speed: decimal(5,3) (nullable = true)
 |-- dew_depression: decimal(6,3) (nullable = true)
 |-- nonideal_temp: double (nullable = true)
 |-- primary_use_idx

In [11]:
# 6a
# Show live predictions
query_live = (
    predictions
        .select("site_id", "building_id", "window", "log_power_usage")
        .writeStream
        .outputMode("append")
        .format("memory")
        .queryName("live_predictions")
        .start()
)
spark.sql("select * from live_predictions").show()


+-------+-----------+------+---------------+
|site_id|building_id|window|log_power_usage|
+-------+-----------+------+---------------+
+-------+-----------+------+---------------+



In [ ]:
# 6b


building_6h = (
    predictions
        .groupBy(
            "building_id",
            "window.start",
            "window.end",
        )
        .agg(F.sum("log_power_usage").alias("total_power_6h"))
)
# --- Print to console every 7 seconds ---
query_building_6h = (
    building_6h
        .writeStream
        .outputMode("update")
        .format("memory")
        .queryName("building_6h")
        .trigger(processingTime="7 seconds")
        .start()
)
spark.sql("select * from building_6h").show()



AnalysisException: Detected pattern of possible 'correctness' issue due to global watermark. The query contains stateful operation which can emit rows older than the current watermark plus allowed late record delay, which are "late rows" in downstream stateful operations and these rows can be discarded. Please refer the programming guide doc for more details. If you understand the possible risk of correctness issue and still need to run the query, you can disable this check by setting the config `spark.sql.streaming.statefulOperator.checkCorrectness.enabled` to false.;
Aggregate [building_id#1, window#779-T5000ms.start, window#779-T5000ms.end], [building_id#1, window#779-T5000ms.start AS start#1235, window#779-T5000ms.end AS end#1236, sum(log_power_usage#1137) AS total_power_6h#1232]
+- Project [site_id#0, building_id#1, primary_use#2, square_feet#3, floor_count#4, row_id#5, year_built#6, latent_y#7, latent_s#8, latent_r#9, date#257, time#268, month#280, window#779-T5000ms, cloud_cover

In [ ]:
# 6c
site_daily = (
    predictions
        .groupBy(
            "site_id", 
            "date"
        .agg(F.sum("log_power_usage").alias("total_power_day"))
)

# --- Print to console every 14 seconds ---
query_site_daily = (
    site_daily
        .writeStream
        .outputMode("update")
        .format("memory")
        .queryName("site_daily")
        .trigger(processingTime="14 seconds")
        .start()
)
spark.sql("select * from site_daily").show()



7.	Save the data from 6 to Parquet files as streams. (Hint: Parquet files support streaming writing/reading. The file keeps updating while new batches arrive.)

In [ ]:
# 7a(save 6a)

# Save predictions to Parquet incrementally
query_live_parquet = (
    predictions
        .select("site_id", "building_id", "time", "log_power_usage")
        .writeStream
        .outputMode("append")
        .format("parquet")
        .option("path", "data/live_predictions")
        .option("checkpointLocation", checkpoint_dir + "/live_predictions")
        .start()
)

In [ ]:
# 7b(save 6b)

query_building_6h_parquet = (
    building_6h
        .writeStream
        .outputMode("update")
        .format("parquet")
        .option("path", "data/building_6h")
        .option("checkpointLocation", checkpoint_dir + "/building_6h")
        .start()
)

In [ ]:
# 7c(save 6c)


query_site_daily_parquet = (
    site_daily
        .writeStream
        .outputMode("update")
        .format("parquet")
        .option("path", "data/site_daily")
        .option("checkpointLocation", checkpoint_dir + "/site_daily")
        .trigger(processingTime="14 seconds")
        .start()
)z

In [ ]:
spark.streams.awaitAnyTermination()

8.	Read the parquet files from task 7 as data streams and send them to Kafka topics with appropriate names.
(Note: You shall read the parquet files as a streaming data frame and send messages to the Kafka topic when new data appears in the parquet file.)

In [ ]:
import json
# Stream 1
live_predictions = (
    spark.readStream
         .format("parquet")
         .schema(predictions.schema)
         .load("data/live_predictions")
)

# send predictions to Kafka
(
    live_predictions
        .selectExpr("CAST(\"predictions\") AS key", 
                    "json.dumps(live_predictions, ensure_ascii=False) AS value")
        .writeStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "localhost:9092")
        .option("topic", "live_predictions")
        .option("checkpointLocation", "checkpoints/predictions_to_kafka")
        .outputMode("append")
        .start()
)


In [ ]:
# Stream 2
building_6h = (
    spark.readStream
         .format("parquet")
         .schema(predictions.schema)
         .load("data/building_6h")
)

# send predictions to Kafka
(
    building_6h
        .selectExpr("CAST(\"predictions\") AS key", 
                    "json.dumps(building_6h, ensure_ascii=False) AS value")
        .writeStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "localhost:9092")
        .option("topic", "building_6h")
        .option("checkpointLocation", "checkpoints/predictions_to_kafka")
        .outputMode("append")
        .start()
)


In [ ]:
# Stream 3
site_daily = (
    spark.readStream
         .format("parquet")
         .schema(predictions.schema)
         .load("data/site_daily")
)

# send predictions to Kafka
(
    site_daily
        .selectExpr("CAST(\"predictions\") AS key", 
                    "json.dumps(site_daily, ensure_ascii=False) AS value")
        .writeStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "localhost:9092")
        .option("topic", "site_daily")
        .option("checkpointLocation", "checkpoints/predictions_to_kafka")
        .outputMode("append")
        .start()
)